# Поляризация каналов

## Терминология

Допустим, у нас есть $n = 2^m$ информационных каналов с равной вероятностью ошибки. Тогда можно произвести процесс поляризации, при котором вместо $n$ физических каналов мы по сути будем передавать информацию в виртуальные каналы. И у половины виртуальных каналов вероятность ошибки будет стремиться к $0$, а у второй половины к $1$ (собственно, поэтому и называется поляризацией).

Обозначим данные, передаваемые физическому каналу, как $c$, а данные, передаваемые виртуальному каналу, как $u$. Тогда поляризация описывается следующей формулой:

$$(c_0 \; c_1 \; \dots \; c_{n-1}) = (u_0 \; u_1 \; \dots \; u_{n-1}) \underset{n \times n}{P} \quad\quad \sim \quad\quad c = u P$$

## Пример

Рассмотрим $2$ двоичных стирающих канала с одинаковой вероятностью ошибки $\mathfrak{C}_0$ и $\mathfrak{C}_1$. Сделаем трюк: вместо просто символов $u_0$ и $u_1$ будем передавать символы $c_0 = u_0 + u_1$ и $c_1 = u_1$. Тогда процесс такой поляризации описывается формулой:

$$(c_0 \; c_1) = (u_0 \; u_1) \begin{pmatrix}1 & 0 \\ 1 & 1\end{pmatrix}$$

Для двоичного стирающего канала можно вычислить вероятности ошибок $P_{error}(u_i)$:
* $u_0$ можно восстановить только если $y_0$ и $y_1$ получены без стираний, т.е. $P_{error}(u_0) = 1 - (1 - p)^2 = 2p - p^2 \ge p$
* Зная $u_0$, можно восстановить $u_1$ только если $y_0$ или $y_1$ получены без стираний, т.е. $P_{error}(u_1) = p^2 \le p$

Ниже рассчитаем аналогичную вероятность для случая $n = 16$:

In [ ]:
from channel.polarization import bec_polar_probability

print(bec_polar_probability(p=0.3, m=4))

## Матрица Адамара

Определим матрицу Адамара $A_m = \begin{pmatrix}1 & 0 \\ 1 & 1\end{pmatrix}^{\otimes m}$, где символ $\otimes m$ обозначает Кронекеровскую степень.

В качестве матрицы поляризации можно выбирать $A_m$, поскольку такая матрица позволяет легко и рекурсивно построить схему соединения каналов. Все дальнейшие вычисления будут допускать, что $P = A_m$.

In [1]:
from algebraic.binary import Binary
from algebraic.matrix import hadamard_matrix

print(hadamard_matrix(3, positive=Binary(1), negative=Binary(0)))

[[1 0 0 0 0 0 0 0]
 [1 1 0 0 0 0 0 0]
 [1 0 1 0 0 0 0 0]
 [1 1 1 1 0 0 0 0]
 [1 0 0 0 1 0 0 0]
 [1 1 0 0 1 1 0 0]
 [1 0 1 0 1 0 1 0]
 [1 1 1 1 1 1 1 1]]


## Максимум правдоподобия в подканалах

На каждом $\lambda$-уровне можно вычислить переходную вероятность $i$-го подканала $W_\lambda^{(i)}$. Для начала определимся с финальным уровнем:

$$W_m(y|c) = \prod_{i = 0}^{n-1} W(y_i, c_i)$$

Тогда можно вычислить $W_m^{(i)}$ для каждого $i$-го подканала:

$$
W_m^{(i)}(y,u_{0 \dots i-1}|u_i) =
\frac{W_m^{(i)}(y,u_{0 \dots i})}{P\{u_i\}} =
2 \sum_{u_{i+1 \dots n-1} \in \mathbb{F}_2^{n-i}} W_m^{(n-1)}(y,u) P\{u\} =
\frac{2}{2^n} \sum_{u_{i+1 \dots n-1} \in \mathbb{F}_2^{n-i}} W_m(y|u A_m) =
\frac{2}{2^n} \sum_{u_{i+1 \dots n-1} \in \mathbb{F}_2^{n-i}} \prod_{j = 0}^{n-1} W(y_j|(u A_m)_j)
$$

И, наконец, рекурсивно определим переходную вероятность для подканалов на $\lambda$-уровне:

$$
W_\lambda^{(2i)}(y_{0 \dots 2^\lambda-1}, u_{0 \dots 2i-1} | u_{2i}) =
\frac{1}{2} \sum_{u_{2i+1} \in \mathbb{F}} W_{\lambda-1}^{(i)}(y_{0, 2 \dots 2^\lambda - 2}, u_{0, 2 \dots 2i - 2} + u_{1, 3 \dots 2i - 1} | u_{2i} + u_{2i+1}) W_{\lambda-1}^{(i)}(y_{1, 3 \dots 2^\lambda - 1}, u_{1, 3 \dots 2i - 1} | u_{2i+1})
$$

$$
W_\lambda^{(2i+1)}(y_{0 \dots 2^\lambda-1}, u_{0 \dots 2i} | u_{2i+1}) =
\frac{1}{2} W_{\lambda-1}^{(i)}(y_{0, 2 \dots 2^\lambda - 2}, u_{0, 2 \dots 2i - 2} + u_{1, 3 \dots 2i - 1} | u_{2i} + u_{2i+1}) W_{\lambda-1}^{(i)}(y_{1, 3 \dots 2^\lambda - 1}, u_{1, 3 \dots 2i - 1} | u_{2i+1})
$$




## Максимум апостериорной вероятности в подканалах

Вычислим $W_m^{(i)}$ для каждого $i$-го подканала:

$$
W_m^{(i)}(u_{0 \dots i}|y) =
\frac{W_m^{(i)}(y,\hat{u}_{0 \dots i-1}|u_i)}{2W(y)} =
\sum_{u_{i+1 \dots n-1} \in \mathbb{F}_2^{n-i}} \prod_{j=0}^{n-1} W((u A_m)_j|y_j)
$$

И, наконец, рекурсивно определим переходную вероятность для подканалов на $\lambda$-уровне:

$$
W_\lambda^{(2i)}( u_{0 \dots 2i}|y) =
\sum_{u_{2i+1} \in \mathbb{F}} W_{\lambda-1}^{(i)}(u_{0, 2 \dots 2i - 2} + u_{1, 3 \dots 2i - 1}|y_{0, 2 \dots n - 2}) W_{\lambda-1}^{(i)}(u_{1, 3 \dots 2i - 1}|y_{1, 3 \dots n - 1})
$$

$$
W_\lambda^{(2i+1)}( u_{0 \dots 2i+1}|y) =
W_{\lambda-1}^{(i)}(u_{0, 2 \dots 2i - 2} + u_{1, 3 \dots 2i - 1}|y_{0, 2 \dots n - 2}) W_{\lambda-1}^{(i)}(u_{1, 3 \dots 2i - 1}|y_{1, 3 \dots n - 1})
$$
